# Benchmarking Our Algorithm against others

In [6]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

   

    # Run the experiment on each algorithm
    for name,alg in algs.items():
        for N in N_range: 

            if executed[name][N]:
                continue
            
            # Construct the environment
            constructor = env_info['constructor']
            specs = env_info['specs']

            print('Building the environment...')
            env = constructor(**specs,N_agent = N)

            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [8]:
from panav.sequential import sequential_HybridSIPP, sequential_planning
from panav.SAMP.solvers import Tube_Planning
from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 
from panav.PIBT import PIBT_plan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP


bloating_r = 0.5
vmax = 1.0

TIMEOUT = 60

# Specify the Algorithms
algs = {  
        "TrafficAware": lambda HG: PIBT_plan(HG,vmax,bloating_r,TIMEOUT,simple_plan=False),
        "NotTrafficAware": lambda HG: PIBT_plan(HG,vmax,bloating_r,TIMEOUT,simple_plan=False,no_traffic_awareness=True),
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [14]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(2,21,2)

In [15]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: MultiTunnel Alg: TrafficAware N agent: 2
################# Time step 0 ################
Remaining agents 2
################# Time step 1 ################
Remaining agents 2
################# Time step 2 ################
Remaining agents 2
################# Time step 3 ################
Remaining agents 2
################# Time step 4 ################
Remaining agents 2
################# Time step 5 ################
Remaining agents 2
################# Time step 6 ################
Remaining agents 2
################# Time step 7 ################
Remaining agents 2
################# Time step 8 ################
Remaining agents 2
################# Time step 9 ################
Remaining agents 2
################# Time step 10 ################
Remaining agents 2
################# Time step 11 ################
Remaining agents 2
################# Time step 12 ################
Remaining agents 2
################# Time step 13 ################
Remaining agents 